In [ ]:
from lambeq import BobcatParser, TreeReader, TreeReaderMode, spiders_reader, cups_reader, stairs_reader
from lambeq import TensorAnsatz, SpiderAnsatz, MPSAnsatz, AtomicType
from discopy import Dim
from classic_pipeline import *
from utilities import *

In [17]:
#define atomic-types
N = AtomicType.NOUN
S = AtomicType.SENTENCE
C = AtomicType.CONJUNCTION
P = AtomicType.PUNCTUATION
NP = AtomicType.NOUN_PHRASE
PP = AtomicType.PREPOSITIONAL_PHRASE

In [18]:
#parser declaration
bobcat_parser = BobcatParser(verbose = "progress")
spider_parser = spiders_reader
cups_parser = cups_reader
stairs_parser = stairs_reader
tree_parser = TreeReader(mode=TreeReaderMode.RULE_ONLY)

In [19]:
#ansatze declaration
tensor_ansatz = TensorAnsatz({N: Dim(2), S: Dim(2), C: Dim(2), P: Dim(2), NP: Dim(2), PP: Dim(2)})
spider_ansatz = SpiderAnsatz({N: Dim(2), S: Dim(2), C: Dim(2), P: Dim(2), NP: Dim(2), PP: Dim(2)})
mps_ansatz = MPSAnsatz({N: Dim(2), S: Dim(2), C: Dim(2), P: Dim(2), NP: Dim(2), PP: Dim(2)}, bond_dim = 3)

In [ ]:
#data-extraction for classic pipeline (linux)

pip = ClassicPipeline(stairs_reader, tensor_ansatz)
train_labels, train_circuits = pip.create_circuits_and_labels("/home/adriano22_/Documents/GitHub/Tesi-Quantum-NLP/project/datasets/edited_datasets/classical/GPS.csv", "n")
test_labels, test_circuits = pip.create_circuits_and_labels("/home/adriano22_/Documents/GitHub/Tesi-Quantum-NLP/project/datasets/edited_datasets/classical/CPN.csv", "n")
eval_labels, eval_circuits = pip.create_circuits_and_labels("/home/adriano22_/Documents/GitHub/Tesi-Quantum-NLP/project/datasets/edited_datasets/classical/ePurse.csv", "n")

In [ ]:
#data-extraction for classic pipeline (win11)
pip = ClassicPipeline(cups_reader, tensor_ansatz)
train_labels, train_circuits = pip.create_circuits_and_labels("C:\\Users\\calif\\Documents\\GitHub\\Tesi-Quantum-NLP\\project\\datasets\\edited_datasets\\classical\\GPS.csv", "n")
test_labels, test_circuits = pip.create_circuits_and_labels("C:\\Users\\calif\\Documents\\GitHub\\Tesi-Quantum-NLP\\project\\datasets\\edited_datasets\\classical\\CPN.csv", "n")
eval_labels, eval_circuits = pip.create_circuits_and_labels("C:\\Users\\calif\\Documents\\GitHub\\Tesi-Quantum-NLP\\project\\datasets\\edited_datasets\\classical\\ePurse.csv", "n")

In [5]:
save_data("train_data.txt", train_labels, train_circuits)
save_data("test_data.txt", test_labels, test_circuits)
save_data("eval_data.txt", eval_labels, eval_circuits)

#train_labels, train_circuits = load_data("train_data.txt")
#test_labels, test_circuits = load_data("test_data.txt")
#eval_labels, eval_circuits = load_data("eval_data.txt")

In [9]:
#train-set (GPS) faulty entries: 127:128 128:129 129:130 136:137 137:138 138:139 147:148 150:151 151:152 155:156 156:157
#test-set (CPN) faulty entries: None
#eval-set (ePurse) faulty entries: 22:23 34:46


rtrain_circuits = train_circuits[0:127] + train_circuits[130:136] + train_circuits[139:147] + train_circuits[148:150] + train_circuits[152:155] + train_circuits[157:]
reval_circuits = eval_circuits[0:22] + eval_circuits[23:34] + eval_circuits[46:]


In [ ]:
"""
working parser/ansatz combos:
    - cups / (tensor, spider, mps) : works on all requirements    
    - stairs / (tensor) : works on all requirements
    - tree / (tensor) : works on all requirements
    - bobcat / (tensor, spider, mps) : works with 90+% of requirements
    
NB: missing combos don't work together, common exception raised is AxiomError   
"""

In [ ]:
#training block for classical pipeline
train_set, test_set, eval_set = pip.create_dataset(train_circuits, train_labels), pip.create_dataset(test_circuits, test_labels), pip.create_dataset(eval_circuits, eval_labels)
model = pip.create_model(train_circuits, test_circuits, eval_circuits)
pip.create_trainer(model = model, loss = torch.nn.BCEWithLogitsLoss(), optimizer = torch.optim.Adam, n_epochs = 100, lr = 3e-2, evaluate = True)
pip.train_and_evaluate(train_set, eval_set, 1, 1)

In [ ]:
epochs = [100]
loss_functions = [torch.nn.BCEWithLogitsLoss(), torch.nn.HingeEmbeddingLoss()]
optimizers = [torch.optim.AdamW, torch.optim.Adagrad, torch.optim.Adam, torch.optim.Adamax]
learning_rates = [3e-1, 3e-2, 3e-3]

train_set, test_set, eval_set = pip.create_dataset(train_circuits, train_labels), pip.create_dataset(test_circuits, test_labels), pip.create_dataset(eval_circuits, eval_labels)
model = pip.create_model(train_circuits, test_circuits, eval_circuits)

for loss in loss_functions:
    for optimizer in optimizers:
        for lr in learning_rates:
            print(f"\nCurrent triple: \n -LEARNING RATE: {str(lr)}\n -LOSS FUNCTION: {str(loss)}\n -OPTIMIZER: {str(optimizer)}")
            pip.create_trainer(model = model, loss = loss, optimizer = optimizer, n_epochs = 100, lr = lr, evaluate = True)
            pip.train_and_evaluate(train_set, eval_set, 1, 1)

Posso rimuovere Adadelta optimizer dalla gridsearcg perchè non dà miglioramenti al training.
Quando si usa HingeEmbeddedLoss ottengo loss value tendenti -infinito.



Dal primo round di gridsearch:
    - con la loss function HingeEmbeddedLoss ottengo sempre valori negativi con conseguenti score bassi nelle metriche di valutazione.
    - con la loss function BCEWithLogitsLoss i valori sono mediamente stabili, le epoch piu equilibrate individuate sono:
            
            Epoch 80:   train/loss: 0.1989   valid/loss: 4.3054   train/acc: 0.8720   train/rec: 0.8571   train/f1: 0.8701   valid/acc: 0.5202   valid/rec: 0.5403   valid/f1: 0.5296     -LEARNING RATE: 0.03  -LOSS FUNCTION: BCEWithLogitsLoss() -OPTIMIZER: <'torch.optim.adamw.AdamW'>

            Epoch 90:   train/loss: 0.3694   valid/loss: 6.0657   train/acc: 0.7649   train/rec: 0.7381   train/f1: 0.7584   valid/acc: 0.5242   valid/rec: 0.4919   valid/f1: 0.5083     -LEARNING RATE: 0.003 -LOSS FUNCTION: BCEWithLogitsLoss() -OPTIMIZER: <'torch.optim.adamw.AdamW'>

            Epoch 95:   train/loss: 0.6820   valid/loss: 0.7440   train/acc: 0.5327   train/rec: 0.4583   train/f1: 0.4952   valid/acc: 0.5000   valid/rec: 0.4435   valid/f1: 0.4701     -LEARNING RATE: 0.003 -LOSS FUNCTION: BCEWithLogitsLoss() -OPTIMIZER: <'torch.optim.adagrad.Adagrad'>

            Epoch 71:   train/loss: 0.2060   valid/loss: 1.2076   train/acc: 0.8571   train/rec: 0.8452   train/f1: 0.8554   valid/acc: 0.5363   valid/rec: 0.5000   valid/f1: 0.5188     -LEARNING RATE: 0.03 -LOSS FUNCTION: BCEWithLogitsLoss() -OPTIMIZER: <'torch.optim.adam.Adam'>


    Le restanti combinazioni non sono riportate perchè hanno o loss troppo alte oppure score delle metriche troppo basse.


In [107]:

import matplotlib.pyplot as plt
from decimal import Decimal

def extract(filename):
    file = open(filename, "r")
    dictionary = {
        "train_loss": list(),
        "test_loss": [],
        "train_acc" : [],
        "test_acc" : [],
        "train_rec" : [],
        "test_rec" : [],
        "train_f1" : [],
        "test_f1" : []
    }
    
    for line in file:
        lista = line.split()
        dictionary["train_loss"] += [float(lista[3])]
        dictionary["test_loss"] += [float(lista[5])]
        dictionary["train_acc"] += [float(lista[7])]
        dictionary["test_acc"] += [float(lista[13])]
        dictionary["train_rec"] += [float(lista[9])]
        dictionary["test_rec"] += [float(lista[15])]
        dictionary["train_f1"] += [float(lista[11])]
        dictionary["test_f1"] += [float(lista[17])]
        
    return dictionary
    
def calculate_precision(f1, rec):
    value =  - ((f1 * rec) / (f1 - 2*rec))
    rounded = round(Decimal(value), 4)
    return rounded
    
def add_precision(diz):
    train_prec = []
    test_prec = []
    
    for item1, item2 in zip(diz["train_rec"], diz["train_f1"]):
        train_prec.append(calculate_precision(item2, item1))
        
    for item1, item2 in zip(diz["test_rec"], diz["test_f1"]):
        test_prec.append(calculate_precision(item2, item1))
        
    diz["train_prec"] = train_prec
    diz["test_prec"] = test_prec
    
    return diz
    
def plot_loss(dictionary):
    x = [i for i in range(0, 100, 1)]
    figure, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 20))
    
    ax1.plot(x, dictionary["train_loss"], label = "train loss")
    ax2.plot(x, dictionary["test_loss"], label = "test loss", color = "orange")
    
    ax1.legend(loc = 'upper right'), ax2.legend(loc = 'upper right')
    ax1.set_xticks([x for x in range(0, 105, 5)]), ax2.set_xticks([x for x in range(0, 105, 5)])
    ax1.set_yticks([y/100 for y in range(0, 100, 5)]), ax2.set_yticks([y/100 for y in range(0, 100, 5)])

    plt.show()
    
def plot_data(dictionary):
    x = [i for i in range(0, 100, 1)]
    figure, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(10, 12))
  
    ax1.plot(x, dictionary["train_acc"], label = "train accuracy")
    ax1.plot(x, dictionary["test_acc"], label = "test accuracy", color = "orange")
    ax4.plot(x, dictionary["train_prec"], label = "train precision")
    ax4.plot(x, dictionary["test_prec"], label = "test precision", color = "orange")
    ax2.plot(x, dictionary["train_rec"], label = "train recall")
    ax2.plot(x, dictionary["test_rec"], label = "test recall", color = "orange")
    ax3.plot(x, dictionary["train_f1"], label = "train f1")
    ax3.plot(x, dictionary["test_f1"], label = "test f1", color = "orange")       
    
    ax1.legend(loc = 'upper right'), ax2.legend(loc = 'upper right'), ax3.legend(loc = 'upper right'), ax4.legend(loc = 'upper right')
    ax1.set_xticks([x for x in range(0, 105, 5)]), ax2.set_xticks([x for x in range(0, 105, 5)]), ax3.set_xticks([x for x in range(0, 105, 5)]), ax4.set_xticks([x for x in range(0, 105, 5)])     
    ax1.set_yticks([y / 100 for y in range(30, 100, 5)]), ax2.set_yticks([y / 100 for y in range(30, 100, 5)]), ax3.set_yticks([y / 100 for y in range(30, 100, 5)]), ax4.set_yticks([y / 100 for y in range(30, 100, 5)])

    plt.show()

In [ ]:
diz = extract("plot.txt") 
plot_data(diz)